## A notebook to download crypto price data locally

In [ ]:
from datetime import datetime

import httpx
import pyarrow as pa

from phidata.table.local.csv import CsvTableLocal
from phidata.checks.not_empty import NotEmpty
from phidata.utils.log import logger
from phidata.utils.print_table import print_table

In [ ]:
# Create CsvTableLocal for storing data
# Path: `storage/tables/crypto_prices`
crypto_prices_local = CsvTableLocal(
    name="crypto_prices",
    database="crypto",
    partitions=["ds"],
    write_checks=[NotEmpty()],
)

### Download prices and load a CsvTableLocal.

In [ ]:
coins = ["bitcoin", "ethereum"]

run_date = datetime.now()
run_day = run_date.strftime("%Y-%m-%d")

logger.info(f"Downloading prices for ds={run_day}")
response = httpx.get(
    url="https://api.coingecko.com/api/v3/simple/price",
    params={
        "ids": ",".join(coins),
        "vs_currencies": "usd",
        "include_market_cap": "true",
        "include_24hr_vol": "true",
        "include_24hr_change": "true",
        "include_last_updated_at": "true",
    },
).json()

# Create pyarrow.Table
# https://arrow.apache.org/docs/python/generated/pyarrow.Table.html#pyarrow.Table.from_pylist
table = pa.Table.from_pylist(
    [
        {
            "ds": run_day,
            "ticker": coin_name,
            "usd": coin_data["usd"],
            "usd_market_cap": coin_data["usd_market_cap"],
            "usd_24h_vol": coin_data["usd_24h_vol"],
            "usd_24h_change": coin_data["usd_24h_change"],
            "last_updated_at": coin_data["last_updated_at"],
        }
        for coin_name, coin_data in response.items()
    ]
)

# Write table to disk
write_success = crypto_prices_local.write_table(table)

In [ ]:
table: pa.Table = crypto_prices_local.read_table()
print_table(title="Crypto Prices", header=table.column_names, rows=table.to_pylist())

In [ ]:
# # Use polars to analyze data
# import polars as pl

# pl_df: pl.DataFrame = pl.DataFrame(table)
# pl_df

In [ ]:
# # Use pandas to analyze data
# import pandas as pd

# pd_df: pd.DataFrame = pd.DataFrame(table)
# pd_df